# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [1]:
# Import packages needed for the project

import json
import pathlib
import urllib.parse

import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db

import psycopg2
from datetime import datetime
from sqlalchemy.orm import declarative_base

## Part 1: Data Preprocessing

In [2]:
# Datafile and path
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

### 1.1 Zipcode Data

In [3]:
def load_and_clean_zipcodes(zipcode_datafile):
    """
    Return a geodataframe.
    
    Keyword arguments:
    zipcode_datafile -- a shape file that contains geometry data
    """
    
    # load data
    df = gpd.read_file(zipcode_datafile)
     
    # Select needed columns
    clean_columns = ['ZIPCODE','geometry']
    df = df[clean_columns]
    
    # Remove invalid data points by rows
    df = df.dropna()
    
    # remove duplicate zip codes
    df.drop_duplicates(subset='ZIPCODE', keep='first', inplace=True)
    
    # Normalize column names
    df.columns = df.columns.str.lower()
    df.rename(columns={'zipcode': 'zip_code'}, inplace=True)

    # Normalize SRID
    df['geometry'] = df['geometry'].to_crs("epsg:4326")
    
    return df


In [4]:
geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)

In [5]:
geodf_zipcode_data

,zip_code,geometry
0,11436,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,11213,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,11218,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."
...,...,...
257,11414,"POLYGON ((-73.85068 40.67165, -73.84995 40.671..."
258,10310,"POLYGON ((-74.12065 40.64104, -74.12057 40.641..."
260,11249,"POLYGON ((-73.95805 40.72442, -73.95772 40.724..."
261,10162,"POLYGON ((-73.95133 40.76931, -73.95165 40.769..."


### 1.2 NYC Open Data: 311 Service Requests Data

In [6]:
def download_and_clean_311_data():
    """
    Download and clean 311 service data, and return a geodataframe.
    """
    
    ### (1)Download the data
    # Set up the url obtained the data from
    url = "https://data.cityofnewyork.us/resource/erm2-nwe9.geojson"
    
    # Select the needed columns to download
    column_select = "?$select=created_date,incident_zip,complaint_type,location"
    url_column = url+column_select
    
    # Set up the headers to pass the application token
    header = {"X-App-Token": "oOu0LSU0TAyxOY44hvVXEv2hT"}

    # Set up the parameter to limit the number of rows and date range
    param = {"$limit": 25000000,
            } 

    # Make the API request and download the geojson file to local
    response = requests.get(url_column, headers=header, params=param)
    
    with open("data/nyc_311.geojson", "wb") as file:
        file.write(response.content)
    
    df = gpd.read_file("data/nyc_311.geojson")
                       
    ### (2)Clean the data
    # Remove invalid data by rows
    df.dropna(inplace=True)
    
    # Normalize column names and types if needed
    df.rename(columns={'incident_zip': 'zip_code'}, inplace=True)
    df['zip_code'] = df['zip_code'].astype("string")
    df['created_date'] = df['created_date'].dt.date
    
    # Normalize SRID
    df['geometry'] = df['geometry'].to_crs("epsg:4326")

                       
    return df

In [7]:
geodf_311_data = download_and_clean_311_data()

In [8]:
geodf_311_data

,created_date,zip_code,complaint_type,geometry
0,2023-12-09,11412,Derelict Vehicles,POINT (-73.75719 40.69898)
1,2023-12-09,11357,Derelict Vehicles,POINT (-73.82518 40.77956)
2,2023-12-09,11222,Derelict Vehicles,POINT (-73.94549 40.71914)
3,2023-12-09,10032,Graffiti,POINT (-73.94337 40.83670)
4,2023-12-09,11211,Graffiti,POINT (-73.95151 40.71341)
...,...,...,...,...
24999995,2014-12-31,10040,HEAT/HOT WATER,POINT (-73.92715 40.86194)
24999996,2014-12-31,10032,HEAT/HOT WATER,POINT (-73.94096 40.82924)
24999997,2014-12-31,11225,HEAT/HOT WATER,POINT (-73.95811 40.66112)
24999998,2014-12-31,10457,PLUMBING,POINT (-73.89439 40.84814)


### 1.3 NYC Open Data: Tree Data

In [9]:
def download_and_clean_tree_data():
    """
    Download and clean tree data, and return a geodataframe.
    """
    
    ### (1)Download the data
    # Set up the url obtained the data from
    url = "https://data.cityofnewyork.us/resource/5rq2-4hqu.geojson"

    # Set up the headers to pass the application token
    header = {"X-App-Token": "oOu0LSU0TAyxOY44hvVXEv2hT"}

    # Set up the parameter to limit the number of rows
    param = {"$limit": 684000} 

    # Make the API request and converting the JSON response to obtain a dataframe
    response = requests.get(url,headers=header, params=param)
    
    with open("data/tree.geojson", "wb") as file:
        file.write(response.content)
        
    df = gpd.read_file("data/tree.geojson")
    
    ### (2)Clean the data
    # Select necessary columns and Remove the others
    clean_columns = ['tree_id','created_at','zipcode','spc_common','health','status','geometry']
    df = df[clean_columns]

    
    # Remove invalid data by rows
    df.dropna(inplace=True)
    
    # Normalize column names and types if needed
    df.rename(columns={'zipcode': 'zip_code'}, inplace=True)
    df.rename(columns={'spc_common': 'species'}, inplace=True)
    df.rename(columns={'created_at': 'created_date'}, inplace=True)
    pd.to_datetime(df['created_date'], format='%m/%d/%Y').dt.date
    
    df['geometry'] = df['geometry'].to_crs("epsg:4326")
    
    
    return df

In [10]:
geodf_tree_data = download_and_clean_tree_data()

In [11]:
geodf_tree_data

,tree_id,created_date,zip_code,species,health,status,geometry
0,180683,08/27/2015,11375,red maple,Fair,Alive,POINT (-73.84422 40.72309)
1,200540,09/03/2015,11357,pin oak,Fair,Alive,POINT (-73.81868 40.79411)
2,204026,09/05/2015,11211,honeylocust,Good,Alive,POINT (-73.93661 40.71758)
3,204337,09/05/2015,11211,honeylocust,Good,Alive,POINT (-73.93446 40.71354)
4,189565,08/30/2015,11215,American linden,Good,Alive,POINT (-73.97598 40.66678)
...,...,...,...,...,...,...,...
683783,155433,08/18/2015,11211,pin oak,Good,Alive,POINT (-73.95494 40.71321)
683784,183795,08/29/2015,11375,Kentucky yellowwood,Good,Alive,POINT (-73.85665 40.71519)
683785,166161,08/22/2015,10314,red maple,Good,Alive,POINT (-74.13652 40.62076)
683786,184028,08/29/2015,10457,red maple,Good,Alive,POINT (-73.90311 40.85083)


### 1.4 Zillow Rent Data

In [12]:
def load_and_clean_zillow_data():
    """
    Load and clean the Zillow rent data, and return a dataframe.
    """
    
    # load the csv file
    df = pd.read_csv(ZILLOW_DATA_FILE)
    
    # Filter out non-NYC cities
    df = df[df['City']=='New York']

    # Remove unused columns
    df.drop(columns=['RegionID', 'SizeRank','RegionType','StateName','State','City','Metro','CountyName'],inplace=True)
    
    # Normalize column names used later
    df.columns = df.columns.str.lower()
    df.rename(columns={'regionname': 'zip_code'}, inplace=True)
    df.rename(columns={'2023-09-30': 'september2023'}, inplace=True)
    df.rename(columns={'2023-08-31': 'august2023'}, inplace=True)
    df.rename(columns={'2023-01-31': 'january2023'}, inplace=True)

    # Normalize the column types
    for col in df.columns:
        if col != 'zip_code':
            df[col] = pd.to_numeric(df[col])
    df['zip_code'] = df['zip_code'].astype("string")
    df1 = df.copy()
    
    # Add a new column 'mean' which computes the average rent for each zipcode within the timeframe for Part 4
    df1['mean'] = df1.iloc[:, 1:].mean(axis=1, skipna=True)
    
    # Select necessary columns
    clean_columns = ['zip_code','january2023','august2023','september2023','mean']# we will round the price in sql
    df1 = df1[clean_columns]
    
    # Remove invalid data by rows
    df1.iloc[:,1:] = df1.iloc[:,1:].fillna(df.iloc[:,1:].mean())
    
    return df1

In [13]:
df_zillow_data=load_and_clean_zillow_data()

In [14]:
df_zillow_data

,zip_code,january2023,august2023,september2023,mean
4,11385,2895.699421,3064.476503,3079.585783,2476.923402
6,11208,2588.030194,2737.547470,2728.733333,2560.469803
12,11236,3029.546370,2285.460026,2362.500000,2323.980013
13,10467,2155.617718,2353.686402,2423.888889,1861.072982
14,11373,2255.604528,2302.557354,2292.994444,2084.549636
...,...,...,...,...,...
6550,10282,7143.347680,7444.236260,7347.458333,7029.339124
6561,11109,4159.759749,4490.367074,4529.358974,3518.929190
6644,10006,3747.416799,4042.831474,4060.096154,3489.762997
6695,10162,3029.546370,4984.693932,5011.666667,4968.739544


### 1.5 Overview of Datasets

In [15]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 248 entries, 0 to 262
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   zip_code  248 non-null    object  
 1   geometry  248 non-null    geometry
dtypes: geometry(1), object(1)
memory usage: 5.8+ KB


In [16]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.geometry

0      POLYGON ((-73.80585 40.68291, -73.80569 40.682...
1      POLYGON ((-73.93740 40.67973, -73.93487 40.679...
2      POLYGON ((-73.90294 40.67084, -73.90223 40.668...
3      POLYGON ((-73.95797 40.67066, -73.95576 40.670...
4      POLYGON ((-73.97208 40.65060, -73.97192 40.650...
                             ...                        
257    POLYGON ((-73.85068 40.67165, -73.84995 40.671...
258    POLYGON ((-74.12065 40.64104, -74.12057 40.641...
260    POLYGON ((-73.95805 40.72442, -73.95772 40.724...
261    POLYGON ((-73.95133 40.76931, -73.95165 40.769...
262    POLYGON ((-73.99354 40.75145, -73.99320 40.751...
Name: geometry, Length: 248, dtype: geometry

In [17]:
geodf_311_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 23680458 entries, 0 to 24999999
Data columns (total 4 columns):
 #   Column          Dtype   
---  ------          -----   
 0   created_date    object  
 1   zip_code        string  
 2   complaint_type  object  
 3   geometry        geometry
dtypes: geometry(1), object(2), string(1)
memory usage: 903.3+ MB


In [18]:
geodf_311_data.head()

,created_date,zip_code,complaint_type,geometry
0,2023-12-09,11412,Derelict Vehicles,POINT (-73.75719 40.69898)
1,2023-12-09,11357,Derelict Vehicles,POINT (-73.82518 40.77956)
2,2023-12-09,11222,Derelict Vehicles,POINT (-73.94549 40.71914)
3,2023-12-09,10032,Graffiti,POINT (-73.94337 40.83670)
4,2023-12-09,11211,Graffiti,POINT (-73.95151 40.71341)


In [19]:
geodf_tree_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 652167 entries, 0 to 683787
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   tree_id       652167 non-null  object  
 1   created_date  652167 non-null  object  
 2   zip_code      652167 non-null  object  
 3   species       652167 non-null  object  
 4   health        652167 non-null  object  
 5   status        652167 non-null  object  
 6   geometry      652167 non-null  geometry
dtypes: geometry(1), object(6)
memory usage: 39.8+ MB


In [20]:
geodf_tree_data.head()

,tree_id,created_date,zip_code,species,health,status,geometry
0,180683,08/27/2015,11375,red maple,Fair,Alive,POINT (-73.84422 40.72309)
1,200540,09/03/2015,11357,pin oak,Fair,Alive,POINT (-73.81868 40.79411)
2,204026,09/05/2015,11211,honeylocust,Good,Alive,POINT (-73.93661 40.71758)
3,204337,09/05/2015,11211,honeylocust,Good,Alive,POINT (-73.93446 40.71354)
4,189565,08/30/2015,11215,American linden,Good,Alive,POINT (-73.97598 40.66678)


In [21]:
df_zillow_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145 entries, 4 to 6721
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   zip_code       145 non-null    string 
 1   january2023    145 non-null    float64
 2   august2023     145 non-null    float64
 3   september2023  145 non-null    float64
 4   mean           145 non-null    float64
dtypes: float64(4), string(1)
memory usage: 6.8 KB


In [92]:
df_zillow_data.head()

,zip_code,january2023,august2023,september2023,mean
4,11385,2895.699421,3064.476503,3079.585783,2476.923402
6,11208,2588.030194,2737.547470,2728.733333,2560.469803
12,11236,3029.546370,2285.460026,2362.500000,2323.980013
13,10467,2155.617718,2353.686402,2423.888889,1861.072982
14,11373,2255.604528,2302.557354,2292.994444,2084.549636


## Part 2: Storing Data

In [23]:
# Assign database name,user,url,and schema
DB_NAME = "Final_Project"
DB_USER = "postgres"
DB_URL = f"postgresql+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"

### 2.1 Creating Database

In [24]:
def setup_new_postgis_database(username, db_name):
    """
    Form and return a connection to the database Final_Project.
    
    Keyword arguments:
    username -- the username for the connection, which will be postgres
    db_name  -- the databse name for the connection, which will be Final_Project
    """
    conn = psycopg2.connect(dbname=db_name, user=username)
    return conn

In [25]:
setup_new_postgis_database(DB_USER, DB_NAME)

<connection object at 0x156e874c0; dsn: 'dbname=Final_Project user=postgres', closed: 0>

### 2.2 Creating Tables

In [26]:
engine = db.create_engine(DB_URL)

In [27]:
ZIPCODE_SCHEMA = """
CREATE TABLE IF NOT EXISTS zipcodes
(
    id INTEGER PRIMARY KEY,
    zip_code CHAR(5),
    geometry GEOMETRY(Polygon,4326)
)
"""

NYC_311_SCHEMA = """
CREATE TABLE IF NOT EXISTS complaints
(
    id INTEGER PRIMARY KEY,
    created_date DATE,
    zip_code CHAR(5),
    complaint_type TEXT,
    geometry GEOMETRY(Point,4326)
)
"""
    
NYC_TREE_SCHEMA = """
CREATE TABLE IF NOT EXISTS trees
(
    id INTEGER PRIMARY KEY,
    tree_id INTEGER,
    created_date DATE,
    zip_code CHAR(5),
    species TEXT,
    health TEXT,
    status TEXT,
    geometry GEOMETRY(Point,4326)
)
"""


ZILLOW_SCHEMA = """
CREATE TABLE IF NOT EXISTS rents
(
    id INTEGER PRIMARY KEY,
    zip_code CHAR(5),
    january2023 FLOAT,
    august2023 FLOAT,
    september2023 FLOAT,
    mean FLOAT
)
"""

In [28]:
# Write the queries schema to the schema file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA + ";\n")
    f.write(NYC_311_SCHEMA + ";\n")
    f.write(NYC_TREE_SCHEMA + ";\n")
    f.write(ZILLOW_SCHEMA + ";\n")

In [29]:
# Execute the schema files to create tables
with engine.connect() as connection:
    connection.execute(open(DB_SCHEMA_FILE, "r").read())

### 2.3 Add Data to Database

In [30]:
def write_dataframes_to_table(tablename_to_dataframe):
    """
    Add data to their respective table.
    
    Keyword arguments:
    tablename_to_dataframe -- a dictionary having table name as the key and data as the value
    """
    
    for table, df in tablename_to_dataframe.items():
        # Add data to table with geospatial data
        if isinstance(df, gpd.GeoDataFrame):
            df.to_postgis(table, con=engine, if_exists='replace', index=False)
            
        # Add data to table without geospatial data
        else:
            df.to_sql(table, con=engine, if_exists='replace', index=False)

In [31]:
tablename_to_dataframe = {
    "zipcodes": geodf_zipcode_data,
    "complaints": geodf_311_data,
    "trees": geodf_tree_data,
    "rents": df_zillow_data,
}

In [32]:
write_dataframes_to_table(tablename_to_dataframe)